In [ ]:
!pip install import-ipynb
import import_ipynb
import sys
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print(drive)

In [ ]:
# Copy the link and remove the front part of the link (i.e. https://drive.google.com/open?id=) to get the file ID.
your_module = drive.CreateFile({'id':'1ezhKHw8J6vMhhi-zA_kGt_6FhPuXgs37'})
your_module.GetContentFile('security_implementation.ipynb')
import security_implementation
import datetime

In [ ]:
_TOTAL_SIZE_IN_BYTES = 0.0

In [ ]:
class Node:
   def __init__(self, data):
      self.left = None
      self.right = None
      self.data = data
      self.encrypted_block = None # Only for leaf nodes
      self.parent = None
      self.timestamp = None
      self.modified = 0
      self.range = [-1, -1] ## Node's range of indices

In [ ]:
def getCurrentTimestamp():
  return datetime.datetime.now()

In [ ]:
def upgradeMerkleTree(prev_node, cur_node, low, high, pos, val):
  #print("low = ", low, "high = ", high, pos, val)
  """
  if cur_node is None or prev_node is None:
    print("OH NO")
  """

  # print("current ",cur_node.range, prev_node.range)
  if (pos > high or pos < low or low > high):
    return None

  # if not prev_node:
  #   return prev_node

  if (low == high):
    # if prev_node is None:
    #   return prev_node
    cur_node.encrypted_block = val
    cur_node.data = security_implementation.calcBlockHash(val.encode('utf-8'))
    # cur_node.range = prev_node.range
    cur_node.range[0] = low
    cur_node.range[1] = high
    cur_node.timestamp = getCurrentTimestamp()
    cur_node.modified = cur_node.modified + 1
    cur_node.left = None
    cur_node.right = None
    #print("$$$$$$$$ Uodated leaf node index: ", low)
    #print(cur_node.encrypted_block, cur_node.data, security_implementation.calcBlockHash(val.encode('utf-8')))
    # print(decrypt_block(cur_node.encrypted_block))

    # cur_node = prev_node
    return

  mid = int((low+high)/2)

  if pos<=mid:
    cur_node.right = prev_node.right
    cur_node.left = Node(0) # left side needs to be updated
    cur_node.left.parent = cur_node
    upgradeMerkleTree(prev_node.left, cur_node.left, low, mid, pos, val)
  else:
    cur_node.left = prev_node.left
    cur_node.right = Node(0) # right side needs to be updated
    cur_node.right.parent = cur_node
    upgradeMerkleTree(prev_node.right, cur_node.right, mid+1, high, pos, val)

  #print("%%%%%%%%")
  #print("low ",low, " high ", high, " mid ", mid)
  #print("left: ", cur_node.left.data)
  #print("right: ", cur_node.right.data)
  assert(cur_node != None)

  cur_node.data = security_implementation.calcBlockHash(cur_node.left.data + cur_node.right.data)
  # if (cur_data.left == None and cur_data.right == None):
  # cur_node.range = prev_node.range
  cur_node.range[0] = low
  cur_node.range[1] = high

  cur_node.timestamp = getCurrentTimestamp()
  cur_node.modified = cur_node.modified + 1 # Check prev vs cur node
  return 

In [ ]:
# For creating the first version
def buildMerkleTree(blocks, cur_node, low, high):
  #Base Case: Leaf Node
  if(low == high):
    cur_node.encrypted_block = blocks[low]
    # print("Leaf node ", blocks[low])
    cur_node.data = security_implementation.calcBlockHash(blocks[low].encode('utf-8')) #Storing encrypted hashed blocks at leaf nodes, converted to unicode
    cur_node.range[0] = low
    cur_node.range[1] = high
    cur_node.timestamp = getCurrentTimestamp()
    # print("#debug: during build: ", low, security_implementation.decrypt_block(cur_node.encrypted_block))
    cur_node.left = None
    cur_node.right = None
    assert(cur_node.encrypted_block == blocks[low])
    return cur_node

  mid = int((low + high)/2);

  # Create and link empty nodes as cur_node's left and right child
  # Save pointer to cur_node at the left and right child
  cur_node.left = Node(0)
  cur_node.right = Node(0)
  cur_node.left.parent = cur_node
  cur_node.right.parent = cur_node

  cur_node.left = buildMerkleTree(blocks, cur_node.left, low, mid)
  cur_node.right = buildMerkleTree(blocks, cur_node.right, mid+1, high)

  # Calculate cur_node's data from left and right child(already calculated by recursion)
  cur_node.data = security_implementation.calcBlockHash(cur_node.left.data + cur_node.right.data) #Hashed
  cur_node.range[0] = low
  cur_node.range[1] = high
  cur_node.timestamp = getCurrentTimestamp()
  return cur_node

In [ ]:
# Bruteforce segment tree for all 
def buildMerkleTreeBruteForce(prev_node, cur_node, low, high, pos, val):
  #Base Case: Leaf Node
  if(low == high):
    cur_node.encrypted_block = prev_node.encrypted_block 
    cur_node.data = prev_node.data # copying nodes 
    cur_node.range[0] = low
    cur_node.range[1] = high
    cur_node.timestamp = getCurrentTimestamp()
    # print("#debug: during build: ", low, security_implementation.decrypt_block(cur_node.encrypted_block))
    cur_node.left = None
    cur_node.right = None

    if low == pos:
      # updating the value 
      cur_node.encrypted_block = val
      cur_node.data = security_implementation.calcBlockHash(val.encode('utf-8'))
    return cur_node

  mid = int((low + high)/2);

  # Create and link empty nodes as cur_node's left and right child
  # Save pointer to cur_node at the left and right child
  cur_node.left = Node(0)
  cur_node.right = Node(0)
  cur_node.left.parent = cur_node
  cur_node.right.parent = cur_node

  cur_node.left = buildMerkleTreeBruteForce(prev_node.left, cur_node.left, low, mid, pos, val)
  cur_node.right = buildMerkleTreeBruteForce(prev_node.right, cur_node.right, mid+1, high, pos, val)

  # Calculate cur_node's data from left and right child(already calculated by recursion)
  cur_node.data = security_implementation.calcBlockHash(cur_node.left.data + cur_node.right.data) #Hashed
  cur_node.range[0] = low
  cur_node.range[1] = high
  cur_node.timestamp = getCurrentTimestamp()
  return cur_node

In [ ]:
def printMerkleTree(root, save_info):
  if root.left is None and root.right is None:
    #print(root.range[0], root.range[1])
    #print(root.encrypted_block)
    save_info[root.range[0]] = root.encrypted_block
    # print(decrypt_block(root.encrypted_block), '\n', decrypt_block(save_info[root.range[0]]), '\n')
    return 
    
  printMerkleTree(root.left, save_info)
  printMerkleTree(root.right, save_info)

In [ ]:
def get_sibling_nodes(node, low, high, pos, sibling_nodes, hashes=True):
  if (pos > high or pos < low or low > high):
    return None
  if node.left == node.right and node.left == None: # reached child node
    return 
  mid = int((low+high)/2)
  if pos<=mid:
    get_sibling_nodes(node.left, low, mid, pos, sibling_nodes, hashes)
    if node.right is not None: 
      if hashes is True:
        sibling_nodes.append(['right',node.right.data])
      else:
        sibling_nodes.append(['right',node.right]) # node version
  else:
    get_sibling_nodes(node.right, mid+1, high, pos, sibling_nodes, hashes)
    if node.left is not None: 
      if hashes is True:
        sibling_nodes.append(['left',node.left.data])
      else:
        sibling_nodes.append(['left',node.left]) # node version
  return  

In [ ]:
def print_sibling_nodes(sibling_nodes):
  for i in range(0, len(sibling_nodes)):
    print("i = ", i, sibling_nodes[i][1].range[0], sibling_nodes[i][1].range[1])

def print_sibling_nodes_hashes(sibling_nodes):
  for i in range(0, len(sibling_nodes)):
    print("i = ", sibling_nodes[i])

In [ ]:
def get_a_leaf_node(node, low, high, pos):
  if(low == high):
    # print(low, high, pos)
    assert(node.range[0] == node.range[1] and node.range[0] == pos)
    return node
  mid = int((low+high)/2)
  if(pos<=mid):
    return get_a_leaf_node(node.left, low, mid, pos)
  else:
    return get_a_leaf_node(node.right, mid+1, high, pos)

In [ ]:
def sanity_test_per_idx(root, base_data):
  # to see, if the encoded strings are properly set 
  # print(root.range, root.left, root.right)
  if root.left is None and root.right is None:
    assert(root.range[0] == root.range[1])
    assert(base_data[root.range[0]] == root.encrypted_block)
    return 
    
  sanity_test_per_idx(root.left, base_data)
  sanity_test_per_idx(root.right, base_data)

In [ ]:
class ServerSegmentTreeControl:
  # a class to control the persistent segment tree 
  def __init__(self):
    self.version_roots = [] # 0 based indexing
    self.max_number_of_leaves = 0

  def build_initial_segment_tree(self, encrypted_blocks):
    self.version_roots = []
    self.max_number_of_leaves=len(encrypted_blocks)
    # print("number of segment tree leaf nodes {}".format(self.max_number_of_leaves))
    # Creating first merkle tree
    v0_root = buildMerkleTree(encrypted_blocks, Node(0), 0, self.max_number_of_leaves-1)
    self.version_roots.append(v0_root)
    return self.version_roots[0]

  def update_segment_tree(self,encrypted_block="", block_to_update=0, bruteforce=False):
    # getting raw blocks
    #raw_blocks = padding_64(block_text)
    #print(raw_blocks)
    # getting encryption 
    #encrypted_blocks = self.do_encryption(raw_blocks=raw_blocks)
    #print(len(encrypted_blocks))
    # new version root
    if bruteforce is False:
      self.version_roots.append(Node(0))
      upgradeMerkleTree(prev_node=self.version_roots[-2], cur_node=self.version_roots[-1], low=0, high=self.max_number_of_leaves-1, 
                              pos=block_to_update, 
                              val=encrypted_block)
    elif bruteforce is True:
      self.version_roots.append(Node(0))
      buildMerkleTreeBruteForce(prev_node=self.version_roots[-2], cur_node=self.version_roots[-1], low=0, high=self.max_number_of_leaves-1, 
                              pos=block_to_update, 
                              val=encrypted_block)
    return self.version_roots[-1]

  def print_tree(self, version_no):
    save_info = {}
    printMerkleTree(root=self.version_roots[version_no], save_info=save_info)

  def file_retrival(self, file_version_no):
    encrypted_blocks = []
    for i in range(0, self.max_number_of_leaves):
      encrypted_blocks.append(get_a_leaf_node(node=self.version_roots[file_version_no], low=0, high=self.max_number_of_leaves-1, pos=i).encrypted_block)
    return encrypted_blocks, self.version_roots[file_version_no].data

In [ ]:
server_seg_tree = ServerSegmentTreeControl()
print(server_seg_tree)

In [ ]:
def initial_file_upload(encrypted_blocks):
    return  server_seg_tree.build_initial_segment_tree(encrypted_blocks=encrypted_blocks)

In [ ]:
def update_existing_file(encrypted_block="", block_to_update=0, bruteforce=False):
    return server_seg_tree.update_segment_tree(encrypted_block, block_to_update, bruteforce)

In [ ]:
def challenge_server(file_version_no, block_index, hashes=True):
    # hashes = True, if I want to recieve only the hashes 
    sibling_nodes = []
    # print("hashes ", hashes)
    get_sibling_nodes(node=server_seg_tree.version_roots[file_version_no], low=0, 
                                      high=server_seg_tree.max_number_of_leaves-1, pos=block_index, sibling_nodes=sibling_nodes, hashes=hashes)
    return sibling_nodes

In [ ]:
def file_retrival_by_client(file_version_no):
  return server_seg_tree.file_retrival(file_version_no=file_version_no)

In [ ]:
def single_object_size(node):
  # sz = sys.getsizeof(node)
  _str =  "Leaf node" if (node.left is None and node.right is None) else "Non leaf node"
  sz = 0
  if _str == "Leaf node":
    sz = len(node.encrypted_block)
  # sz = len(node.data) + sz + 4*2
  sz = 60 # sys.getsizeof(node) # node.__sizeof__()
  # print("sz ",sz,_str)
  """
  sz = 0
  sz = sz + sys.getsizeof(node.left)
  sz = sz + sys.getsizeof(node.right)
  sz = sz + sys.getsizeof(node.data)
  sz = sz + sys.getsizeof(node.encrypted_block)
  sz = sz + sys.getsizeof(node.parent)
  #sz = sz + sys.getsizeof(node.timestamp)
  #sz = sz + sys.getsizeof(node.modified)
  sz = sz + sys.getsizeof(node.range)
  """
  return sz

In [ ]:
def recursive_size_check(node, _dict):
  global _TOTAL_SIZE_IN_BYTES
  if _dict.get(node) is None:
    _dict[node] = True 
    _TOTAL_SIZE_IN_BYTES = _TOTAL_SIZE_IN_BYTES + single_object_size(node)
  else:
    return 
  if node.left is not None:
    recursive_size_check(node.left, _dict)
  if node.right is not None:
    recursive_size_check(node.right, _dict)